Function to find root of this project. This is not required when using the code normally

In [ ]:
from pathlib import Path
import os

root = Path.cwd()

while not((root / 'requirements.txt').exists() and (root / 'README.md').exists()):
    root = root.parent

root = root / 'src'

os.chdir(root)

# Start of actual code

In [ ]:
import numpy as np
from skopt.learning import GaussianProcessRegressor
from skopt.learning.gaussian_process.kernels import Matern, WhiteKernel, RBF, RationalQuadratic
import matplotlib.pyplot as plt

from ml_utils.design_eval.bayes_opt import suggest_next_experiment

In [ ]:
import warnings
from sklearn.exceptions import ConvergenceWarning

warnings.filterwarnings('ignore', category=ConvergenceWarning) # related to reducing the minimum noise, which we wish to avoid

# example of Bayesian Optimization in a 1D system

In [ ]:
# import the data
X = np.array([[0.1], [0.3], [0.6], [0.8]])
y = np.array([-0.5, -0.2, 0.4, 0.1])

# fit the model
kernel = Matern(length_scale=1.0, nu=2.5)
gp = GaussianProcessRegressor(
    kernel=kernel,
    normalize_y=True,
    n_restarts_optimizer=5,
    random_state=0)

gp.fit(X, y)

# define the candidate space
X_candidates = np.linspace(0.1,0.9,100).reshape(-1, 1)

# use Bayesian Optimization to recommend 3 experiments
acq, i_suggested, x_suggested = suggest_next_experiment(X,
                                                        y,
                                                        X_candidates,
                                                        goal='max',
                                                        model_actual=gp,
                                                        acq_func_name='EI',
                                                        n_suggestions=3,
                                                        kriging_believer='prediction',
                                                        xi=0.1)

# visualize the results
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(X, y)

y_pred, y_std = gp.predict(X_candidates, return_std=True)

X_candidates_1d = X_candidates.flatten()
ax.plot(X_candidates_1d, y_pred)
ax.fill_between(X_candidates_1d, y_pred - y_std, y_pred + y_std, alpha=0.5)

ax2 = ax.twinx()
ax2.plot(X_candidates_1d, acq[0], c='k')
for x in x_suggested:
    ax.axvline(x, c='k', ls=':')

ax.set_xlabel('x')
ax.set_ylabel('y')
ax2.set_ylabel('acq. func. value')

# more complex example

## pretend that this is our exact system

In [ ]:
def my_system(x):
    return np.sin(x) - np.exp(-x**2) + np.log(x**2) + 1

fig = plt.figure()
ax = fig.add_subplot(111)

X = np.linspace(-10, 10, 200)
y = my_system(X)
ax.plot(X, y)
ax.set_xlabel('x')
ax.set_ylabel('y')

In [ ]:
# let's start with 4 evenly spaced points
x_min_design, x_max_design, n_data = -10, 10, 4
X = np.linspace(x_min_design, x_max_design, n_data).reshape(-1, 1)
y = my_system(X).flatten()

fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(X.flatten(), y)
ax.set_title('Our understanding of the space with just 4 examples')
ax.set_xlabel('x')
ax.set_ylabel('y')

# use Bayes opt to find the global maximum
## start with the basic model

In [ ]:
# fit the model
kernel = RationalQuadratic() + WhiteKernel(0.1)

gp = GaussianProcessRegressor(
    kernel=kernel,
    normalize_y=True,
    n_restarts_optimizer=100,
    random_state=0)

gp.fit(X, y)

# define the candidate space
X_candidates = np.linspace(x_min_design, x_max_design, 100).reshape(-1, 1)

# use Bayesian Optimization to recommend 3 experiments
acq, i_suggested, X_suggested = suggest_next_experiment(X,
                                                        y,
                                                        X_candidates,
                                                        goal='max',
                                                        model_actual=gp,
                                                        acq_func_name='EI',
                                                        n_suggestions=2,
                                                        kriging_believer='prediction',
                                                        xi=1)

# visualize the results
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(X, y)

y_pred, y_std = gp.predict(X_candidates, return_std=True)

X_candidates_1d = X_candidates.flatten()
ax.plot(X_candidates_1d, y_pred)
ax.fill_between(X_candidates_1d, y_pred - y_std, y_pred + y_std, alpha=0.5)

ax2 = ax.twinx()
for x in X_suggested:
    ax.axvline(x, c='k', ls=':')

ax.set_xlabel('x')
ax.set_ylabel('y')
ax2.set_ylabel('acq. func. value')


## update the model and try again

In [ ]:
X = np.vstack([X, X_suggested])
y = my_system(X).flatten()

gp.fit(X, y)

# use Bayesian Optimization to recommend 3 experiments
acq, i_suggested, X_suggested = suggest_next_experiment(X,
                                                        y,
                                                        X_candidates,
                                                        goal='max',
                                                        model_actual=gp,
                                                        acq_func_name='EI',
                                                        n_suggestions=1,
                                                        kriging_believer='prediction',
                                                        xi=0.1)

# visualize the results
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(X, y)

y_pred, y_std = gp.predict(X_candidates, return_std=True)

X_candidates_1d = X_candidates.flatten()
ax.plot(X_candidates_1d, y_pred)
ax.fill_between(X_candidates_1d, y_pred - y_std, y_pred + y_std, alpha=0.5)

ax2 = ax.twinx()
for x in X_suggested:
    ax.axvline(x, c='k', ls=':')

ax.set_xlabel('x')
ax.set_ylabel('y')
ax2.set_ylabel('acq. func. value')

## update the model and try again

In [ ]:
X = np.vstack([X, X_suggested])
y = my_system(X).flatten()

gp.fit(X, y)

# use Bayesian Optimization to recommend 3 experiments
acq, i_suggested, X_suggested = suggest_next_experiment(X,
                                                        y,
                                                        X_candidates,
                                                        goal='max',
                                                        model_actual=gp,
                                                        acq_func_name='EI',
                                                        n_suggestions=1,
                                                        kriging_believer='prediction',
                                                        xi=0.1)

# visualize the results
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(X, y)

y_pred, y_std = gp.predict(X_candidates, return_std=True)

X_candidates_1d = X_candidates.flatten()
ax.plot(X_candidates_1d, y_pred)
ax.fill_between(X_candidates_1d, y_pred - y_std, y_pred + y_std, alpha=0.5)

ax2 = ax.twinx()
for x in X_suggested:
    ax.axvline(x, c='k', ls=':')

ax.set_xlabel('x')
ax.set_ylabel('y')
ax2.set_ylabel('acq. func. value')

## update once more and compare with ground truth
we see that we got a really good maximum

In [ ]:
X = np.vstack([X, X_suggested])
y = my_system(X).flatten()

gp.fit(X, y)

# visualize the results
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(X, y)

y_pred, y_std = gp.predict(X_candidates, return_std=True)

X_candidates_1d = X_candidates.flatten()
ax.plot(X_candidates_1d, y_pred)
ax.fill_between(X_candidates_1d, y_pred - y_std, y_pred + y_std, alpha=0.5)

ax.set_xlabel('x')
ax.set_ylabel('y')

X_truth = np.linspace(x_min_design, x_max_design, 200)
y_truth = my_system(X_truth)
ax.plot(X_truth, y_truth, c='k', ls=':')
ax.set_xlabel('x')
ax.set_ylabel('y')

# save the final max from Bayes Opt
i = np.argmax(y)
bayes_opt_best_max = (X[i], y[i])
ax.scatter(bayes_opt_best_max[0], bayes_opt_best_max[1], c='k', marker='x')

# now, let's try to find the global minimum
## reset the design, startign witht the same data

In [ ]:
X = np.linspace(x_min_design, x_max_design, n_data).reshape(-1, 1)
y = my_system(X).flatten()

fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(X.flatten(), y)
ax.set_title('Our understanding of the space with just 4 examples')
ax.set_xlabel('x')
ax.set_ylabel('y')

## let's do the same, but now we minize

In [ ]:
# fit the model
kernel = RationalQuadratic() + WhiteKernel(0.1)

gp = GaussianProcessRegressor(
    kernel=kernel,
    normalize_y=True,
    n_restarts_optimizer=100,
    random_state=0)

gp.fit(X, y)

# define the candidate space
X_candidates = np.linspace(x_min_design, x_max_design, 100).reshape(-1, 1)

# use Bayesian Optimization to recommend 3 experiments
acq, i_suggested, X_suggested = suggest_next_experiment(X,
                                                        y,
                                                        X_candidates,
                                                        goal='min', # CHANGED TO MIN
                                                        model_actual=gp,
                                                        acq_func_name='EI',
                                                        n_suggestions=2,
                                                        kriging_believer='prediction',
                                                        xi=1)

# visualize the results
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(X, y)

y_pred, y_std = gp.predict(X_candidates, return_std=True)

X_candidates_1d = X_candidates.flatten()
ax.plot(X_candidates_1d, y_pred)
ax.fill_between(X_candidates_1d, y_pred - y_std, y_pred + y_std, alpha=0.5)

ax2 = ax.twinx()
for x in X_suggested:
    ax.axvline(x, c='k', ls=':')

ax.set_xlabel('x')
ax.set_ylabel('y')
ax2.set_ylabel('acq. func. value')


## let's repeat 3x

In [ ]:
for _ in range(3):
    X = np.vstack([X, X_suggested])
    y = my_system(X).flatten()

    gp.fit(X, y)

    # use Bayesian Optimization to recommend 3 experiments
    acq, i_suggested, X_suggested = suggest_next_experiment(X,
                                                            y,
                                                            X_candidates,
                                                            goal='min',
                                                            model_actual=gp,
                                                            acq_func_name='EI',
                                                            n_suggestions=1,
                                                            kriging_believer='prediction',
                                                            xi=0.1)

    # visualize the results
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.scatter(X, y)

    y_pred, y_std = gp.predict(X_candidates, return_std=True)

    X_candidates_1d = X_candidates.flatten()
    ax.plot(X_candidates_1d, y_pred)
    ax.fill_between(X_candidates_1d, y_pred - y_std, y_pred + y_std, alpha=0.5)

    ax2 = ax.twinx()
    for x in X_suggested:
        ax.axvline(x, c='k', ls=':')

    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax2.set_ylabel('acq. func. value')

## let's see how good our model did

In [ ]:
# visualize the results
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(X, y)

y_pred, y_std = gp.predict(X_candidates, return_std=True)

X_candidates_1d = X_candidates.flatten()
ax.plot(X_candidates_1d, y_pred)
ax.fill_between(X_candidates_1d, y_pred - y_std, y_pred + y_std, alpha=0.5)

ax.set_xlabel('x')
ax.set_ylabel('y')

X_truth = np.linspace(x_min_design, x_max_design, 200)
y_truth = my_system(X_truth)
ax.plot(X_truth, y_truth, c='k', ls=':')
ax.set_xlabel('x')
ax.set_ylabel('y')

i = np.argmin(y)
bayes_opt_best_min = (X[i], y[i])

# but why not just take x points evenly spaced?

So let's try to fit a model that took 8 evenly spaced points

In [ ]:
# get the same number of points as Bayes opt above but evenly spaced
X = np.linspace(x_min_design, x_max_design, X.shape[0]).reshape(-1, 1)
y = my_system(X).flatten()

# fit the model
kernel = RationalQuadratic() + WhiteKernel(0.1)

gp = GaussianProcessRegressor(
    kernel=kernel,
    normalize_y=True,
    n_restarts_optimizer=100,
    random_state=0)

gp.fit(X, y)

# visualize the results
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(X, y)

y_pred, y_std = gp.predict(X_candidates, return_std=True)

X_candidates_1d = X_candidates.flatten()
ax.plot(X_candidates_1d, y_pred)
ax.fill_between(X_candidates_1d, y_pred - y_std, y_pred + y_std, alpha=0.5)

ax.set_xlabel('x')
ax.set_ylabel('y')

X_truth = np.linspace(x_min_design, x_max_design, 200)
y_truth = my_system(X_truth)
ax.plot(X_truth, y_truth, c='k', ls=':')
ax.set_xlabel('x')
ax.set_ylabel('y')

# plot the global min and global max of the evenly spread set
i = np.argmin(y)
ax.scatter(X[i], y[i], c='aqua', marker='*')
i = np.argmax(y)
ax.scatter(X[i], y[i], c='aqua', marker='*')

# plot the min and max from Bayes Opt
ax.scatter(bayes_opt_best_max[0], bayes_opt_best_max[1], c='k', marker='x')
ax.scatter(bayes_opt_best_min[0], bayes_opt_best_min[1], c='k', marker='x')

# plot the true min and max
i = np.argmin(y_truth)
ax.scatter(X_truth[i], y_truth[i], c='r', marker='+')
i = np.argmax(y_truth)
ax.scatter(X_truth[i], y_truth[i], c='r', marker='+')

We see that, at worst, Bayes Opt is approx. comparable to evenly spaced experimentation. At best, Bayes Opt is approx. better than evenly spaced experimentation. However, when it comes to getting a general idea of the entire space, evenly spaced experiments are definitely superior.